In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/skin-cancer-mnist-ham10000/hmnist_8_8_RGB.csv
/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv
/kaggle/input/skin-cancer-mnist-ham10000/hmnist_8_8_L.csv
/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_L.csv
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0028933.jpg
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0028394.jpg
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0027799.jpg
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0028100.jpg
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0027960.jpg
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0028872.jpg
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0026412.jpg
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0024872.jpg
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000

In [2]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [5]:
df = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv', delimiter = ',')
df.dataframeName = 'hmnist_28_28_RGB.csv'
nRow, nCol = df.shape
print(f'{nRow} rows and {nCol} columns')

10015 rows and 2353 columns


In [9]:
# torch tool
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

# seed
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [43]:
target = df['label']
data = df.iloc[:,:-1]

final_Lst = []
for i in data.index:
    Lst = []
    Lst.append(data.loc[i,:].values)
    
    final_Lst.append(Lst)

In [44]:
image_data = np.array(final_Lst).reshape(-1,28,28,3)

In [47]:
image_data = image_data.astype('float')

In [48]:
mean_value = []
std_value = []
for i in range(3):
    RGB = image_data[:,:,:,i].ravel() # 평평하게 만든다
    
    image_data[:,:,:,i]=(image_data[:,:,:,i] - RGB.mean())/RGB.std()
    
image_data

array([[[[-0.07664671,  0.35765664,  1.11332941],
         [ 0.00792499,  0.40979205,  1.08988952],
         [ 0.06430611,  0.38372435,  0.92581028],
         ...,
         [-0.04845614,  0.12304727,  0.17573375],
         [ 0.06430611,  0.38372435,  0.41013267],
         [-0.02026558,  0.33158893,  0.33981299]],

        [[ 0.06430611,  0.592266  ,  1.32428843],
         [ 0.17706837,  0.54013059,  1.20708898],
         [ 0.20525893,  0.54013059,  1.11332941],
         ...,
         [-0.02026558,  0.20125039,  0.08197419],
         [ 0.09249668,  0.48799517,  0.48045234],
         [-0.07664671,  0.35765664,  0.50389223]],

        [[ 0.06430611,  0.40979205,  1.11332941],
         [ 0.09249668,  0.38372435,  0.92581028],
         [ 0.20525893,  0.43585976,  1.06644963],
         ...,
         [-0.02026558,  0.25338581,  0.15229386],
         [ 0.12068724,  0.5661983 ,  0.73829115],
         [-0.02026558,  0.43585976,  0.69141136]],

        ...,

        [[-0.02026558,  0.46192747,  1

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image_data, target, test_size = 0.2)

print(X_train.shape, X_test.shape)

(8012, 28, 28, 3) (2003, 28, 28, 3)


In [99]:
import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
from keras.applications.resnet50 import ResNet50
from keras import backend as K 

ModuleNotFoundError: No module named 'keras'

## CNN

In [108]:
input_shape = (28,28,3)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(64, kernel_size = (2,2), padding = 'same', input_shape = input_shape))
model.add(MaxPool2D(pool_size = (3,3)))
model.add(Dropout(0.3))

model.add(Conv2D(32, kernel_size = (2,2), padding = 'same'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(16, kernel_size = (2,2), padding = 'same'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(7, activation = 'softmax'))

print(model.summary())

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 28, 28, 64)        832       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 9, 9, 64)          0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 9, 9, 32)          8224      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 4, 4, 16)        

In [109]:
model.compile(optimizer = Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [110]:
model.fit(X_train, y_train, epochs = 50, batch_size = 28, validation_data = (X_test, y_test))

Epoch 1/50
287/287 [==============================] - 2s 5ms/step - loss: 1.0376 - accuracy: 0.6523 - val_loss: 0.8577 - val_accuracy: 0.6965
Epoch 2/50
287/287 [==============================] - 1s 4ms/step - loss: 0.8461 - accuracy: 0.7021 - val_loss: 0.7917 - val_accuracy: 0.7094
Epoch 3/50
287/287 [==============================] - 1s 4ms/step - loss: 0.8049 - accuracy: 0.7085 - val_loss: 0.7466 - val_accuracy: 0.7159
Epoch 4/50
287/287 [==============================] - 1s 4ms/step - loss: 0.7675 - accuracy: 0.7237 - val_loss: 0.7361 - val_accuracy: 0.7179
Epoch 5/50
287/287 [==============================] - 1s 4ms/step - loss: 0.7550 - accuracy: 0.7206 - val_loss: 0.7522 - val_accuracy: 0.7274
Epoch 6/50
287/287 [==============================] - 1s 4ms/step - loss: 0.7372 - accuracy: 0.7199 - val_loss: 0.7574 - val_accuracy: 0.7189
Epoch 7/50
287/287 [==============================] - 1s 5ms/step - loss: 0.7016 - accuracy: 0.7448 - val_loss: 0.7664 - val_accuracy: 0.7199
Epoch 

In [111]:
from sklearn.metrics import accuracy_score

y_pred = model.predict_classes(X_test)
# RGB로 scaling을 해줬을 때 75.4%로 가장 좋은 성능을 낸다.
np.mean(y_pred == y_test)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.7698452321517724

## VGG16

In [114]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

Model_input = 224

# 논문에 나온대로 image를 (-1, 224, 224, 3)으로 변경한다.
image_data = tf.image.resize(image_data, [Model_input, Model_input], method = 'nearest')

In [123]:
input_shape = (224,224,3)

# VGG16도 안돌아간다?
model = Sequential()
model.add(Conv2D(64, kernel_size = (3,3), input_shape = input_shape, padding = 'same'))
model.add(Conv2D(128, kernel_size = (3,3), padding = 'same'))
model.add(Flatten())
model.add(Dense(4096, activation = 'relu'))
model.add(Dense(1000, activation = 'relu'))
model.add(7, activation = 'softmax')

2021-09-17 18:56:19.417437: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 98.00GiB (rounded to 105226698752)requested by op RandomUniform
Current allocation summary follows.
2021-09-17 18:56:19.417535: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2021-09-17 18:56:19.417561: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 313, Chunks in use: 313. 78.2KiB allocated for chunks. 78.2KiB in use in bin. 17.9KiB client-requested in use in bin.
2021-09-17 18:56:19.417577: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 27, Chunks in use: 27. 13.8KiB allocated for chunks. 13.8KiB in use in bin. 13.5KiB client-requested in use in bin.
2021-09-17 18:56:19.417592: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 45, Chunks in use: 44. 62.0KiB allocated for chunks. 61.0KiB in use in bin. 60.5K

ResourceExhaustedError: OOM when allocating tensor with shape[6422528,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [ ]:
model.summary()